In [1]:
%%capture
!pip install transformers[sentencepiece]
import transformers

Processing the data.

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# PATH_TO_FOLDER = "drive/MyDrive/Name2Demographics/"
PATH_TO_FOLDER = "../../../"
%cd $PATH_TO_FOLDER

/home/souvic/mounted/btp/vahini/Name2Demographics


In [4]:
import pandas as pd
import numpy as np

import sys  

sys.path.insert(0, 'Models/AIEEEDataState/PreProcessing/')
from aieee_caste import *

In [5]:
%%capture
!pip install datasets

In [6]:
caste_df = AIEEECasteStateFinalData()

/home/souvic/mounted/btp/vahini/Name2Demographics/Models/AIEEEDataState/PreProcessing/aieee_caste.py:251: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['category'] = df['category'].replace('', np.NaN)


In [7]:
caste_df.head()

,Name,Caste
0,gauri shankar pandey 35,0.0
1,nelluri teja bharath 14,0.0
2,jerin t jerome 04,1.0
3,bhuri gurjar 16,1.0
4,abhishek deopa 27,0.0


In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

In [9]:
from sklearn.model_selection import train_test_split

X = list(caste_df["Name"])
y = list(map(int,caste_df["Caste"]))
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, return_tensors="pt")
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, return_tensors="pt")

In [10]:
caste_df.groupby('Caste').count()/caste_df.shape[0] * 100

,Name
Caste,
0.0,58.172387
1.0,41.827613


In [11]:
import torch

# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [12]:
from transformers import TrainingArguments
from transformers import Trainer

In [13]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import EarlyStoppingCallback


# Define Trainer parameters
def compute_metric(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
#     print('here')

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    g_recall = recall_score(y_true=labels, y_pred=pred, pos_label=0)
    g_precision = precision_score(y_true=labels, y_pred=pred, pos_label=0)
    g_f1 = f1_score(y_true=labels, y_pred=pred,pos_label=0)
    r_recall = recall_score(y_true=labels, y_pred=pred, pos_label=1)
    r_precision = precision_score(y_true=labels, y_pred=pred, pos_label=1)
    r_f1 = f1_score(y_true=labels, y_pred=pred,pos_label=1)
    return {"accuracy": accuracy, "general_precision": g_precision, "general_recall":g_recall, "general_f1": g_f1,
        "reserved_precision": r_precision, "reserved_recall": r_recall, "reserved_f1": r_f1 }

# Define Trainer
args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=100,
    per_device_train_batch_size=50,
    per_device_eval_batch_size=50,
    num_train_epochs=1,
    seed=42,
    load_best_model_at_end=True,
#     learning_rate=1e-03,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metric,
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

# Train pre-trained model
trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vahini10 (use `wandb login --relogin` to force relogin)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


/tmp/ipykernel_20648/3554856749.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/home/souvic/.conda/envs/py38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Accuracy,General Precision,General Recall,General F1,Reserved Precision,Reserved Recall,Reserved F1
100,No log,0.626005,0.652258,0.729234,0.640422,0.681949,0.571745,0.668746,0.616453
200,No log,0.585420,0.669315,0.782325,0.598445,0.678141,0.578593,0.768040,0.659990
300,No log,0.550861,0.698473,0.755332,0.712967,0.733538,0.629125,0.678281,0.652779
400,No log,0.535417,0.710143,0.756663,0.740071,0.748275,0.648638,0.668452,0.658396
500,0.578200,0.534894,0.718555,0.725171,0.831735,0.774806,0.705262,0.560889,0.624844
600,0.578200,0.546559,0.710672,0.810344,0.656667,0.725457,0.621668,0.785903,0.694204
700,0.578200,0.514151,0.724679,0.729057,0.838749,0.780066,0.715802,0.565774,0.632006
800,0.578200,0.524000,0.725429,0.722544,0.857677,0.784332,0.731882,0.541201,0.622261
900,0.578200,0.510023,0.733152,0.758162,0.795272,0.776274,0.693933,0.646616,0.669439
1000,0.517500,0.514933,0.735464,0.754428,0.808856,0.780695,0.703975,0.633225,0.666729


/tmp/ipykernel_20648/3554856749.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/home/souvic/.conda/envs/py38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/tmp/ipykernel_20648/3554856749.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/home/souvic/.conda/envs/py38/lib/python3.8/site-packages/torch/nn/parallel/_funct

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

/tmp/ipykernel_20648/3554856749.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/home/souvic/.conda/envs/py38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/tmp/ipykernel_20648/3554856749.py:10: UserWarning: To copy construct 

TrainOutput(global_step=1898, training_loss=0.5229961367125757, metrics={'train_runtime': 6964.1803, 'train_samples_per_second': 27.245, 'train_steps_per_second': 0.273, 'total_flos': 1852570593566340.0, 'train_loss': 0.5229961367125757, 'epoch': 1.0})

In [14]:
metrics=trainer.evaluate()
print(metrics)

/tmp/ipykernel_20648/3554856749.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'eval_loss': 0.4844941198825836, 'eval_accuracy': 0.7470362536278223, 'eval_general_precision': 0.7824609539890249, 'eval_general_recall': 0.7831882710833192, 'eval_general_f1': 0.7828244435998141, 'eval_reserved_precision': 0.697577793493635, 'eval_reserved_recall': 0.6966745144202472, 'eval_reserved_f1': 0.6971258613581482, 'eval_runtime': 164.7663, 'eval_samples_per_second': 493.523, 'eval_steps_per_second': 4.94, 'epoch': 1.0}


In [15]:
metrics

{'eval_loss': 0.4844941198825836,
 'eval_accuracy': 0.7470362536278223,
 'eval_general_precision': 0.7824609539890249,
 'eval_general_recall': 0.7831882710833192,
 'eval_general_f1': 0.7828244435998141,
 'eval_reserved_precision': 0.697577793493635,
 'eval_reserved_recall': 0.6966745144202472,
 'eval_reserved_f1': 0.6971258613581482,
 'eval_runtime': 164.7663,
 'eval_samples_per_second': 493.523,
 'eval_steps_per_second': 4.94,
 'epoch': 1.0}

## Save the Model

In [16]:
torch.save(model.state_dict(), "Models/AIEEEData/MurilBERT/SavedModel/MurilBERT")

In [16]:
model.save_pretrained("Models/AIEEEDataState/BERTWithState/SavedModel/BERT")